In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
import scipy.stats as st
from pathlib import Path


In [2]:
#pull in CSV df - 2019 through 2021 - vehicle csv

In [3]:
#clean data for CA (Legal) -  years 2019 through 2021 - MONTHNAME - ACC_TYPE - OWNER - MAK_MODNAME- BODY_TYPNAME - HARM_EVNAME - MAN_COLLNAME - VNUM_LAN(exculed 1 lane)

#legal_2019_df = cleaned2019.loc[(cleaned_2019['STATE']== "6")]


In [4]:
#clean data for (Somewhat Legal) - years 2019 through 2021 - MONTHNAME - ACC_TYPE - OWNER - MAK_MODNAME- BODY_TYPNAME - HARM_EVNAME - MAN_COLLNAME - VNUM_LAN(exculed 1 lane)

#somewhat_legal_2019_df = cleaned_2019.loc[cleaned_2019['STATE'].isin([15,49])]

In [5]:
#clean data for (Illegal) - years 2019 through 2021 - MONTHNAME - ACC_TYPE - OWNER - MAK_MODNAME- BODY_TYPNAME - HARM_EVNAME - MAN_COLLNAME - VNUM_LAN(exculed 1 lane)

#illegal_2019_df = cleaned_2019.loc[~cleaned_2019['STATE'].isin([6,15,49,30,35,48,40,29,5,28,20,39,54,37,34,43,52])] 

In [6]:
#clean data for (Not Mentioned) - years 2019 through 2021 - MONTHNAME - ACC_TYPE - OWNER - MAK_MODNAME- BODY_TYPNAME - HARM_EVNAME - MAN_COLLNAME - VNUM_LAN(exculed 1 lane)

#not_mentioned__2019df = cleaned_2019.loc[cleaned_2019['STATE'].isin([30,35,48,40,29,5,28,20,39,54,37,34])] 

In [25]:
# Data of motorcycle registration numbers cleaned and displayed

motorcycle_df = pd.read_csv("Resources/motorcycles.csv")

motorcycle_clean= motorcycle_df.drop(0)
column_names = motorcycle_clean.iloc[0].fillna("State")
motorcycle_clean.columns = column_names
motorcycle_clean = motorcycle_clean.drop([1,54,55])
motorcycles_df = motorcycle_clean[["State", "2019","2020","2021"]]
motor_reg= motorcycles_df.copy()

motor_reg[["2019", "2020","2021"]] = motor_reg[["2019", "2020","2021"]].astype(str).apply(lambda x: x.str.replace(',', ''))
motor_reg[["2019", "2020","2021"]] = motor_reg[["2019", "2020","2021"]].astype(float)


motor_reg 



1,State,2019,2020,2021
2,Alabama,173652.0,171077.0,167021.0
3,Alaska,24340.0,23918.0,23068.0
4,Arizona,212951.0,216705.0,222484.0
5,Arkansas,71764.0,70586.0,72474.0
6,California,957620.0,957873.0,952977.0
7,Colorado,189449.0,202474.0,201364.0
8,Connecticut,91296.0,89805.0,88061.0
9,Delaware,26571.0,26594.0,26729.0
10,District of Columbia,5290.0,5254.0,5020.0
11,Florida,601849.0,620892.0,645012.0


In [26]:
# Motorcycles that are registered split up by laws

legal_reg = motor_reg.loc[motor_reg["State"] =="California"]

somewhat_legal_reg = motor_reg.loc[motor_reg["State"].isin(["Utah","Hawaii"])]

not_mentioned_reg= motor_reg.loc[motor_reg["State"].isin(["Montana", "New Mexico", "Texas", "Oklahoma",
                                                                  "Missouri", "Arkansas", "Ohio", "Kentucky",
                                                                  "North Carolina", "West Virgina", "Delaware","Mississippi"])]

illegal_reg = motor_reg.loc[~motor_reg["State"].isin(["Montana", "New Mexico", "Texas", "Oklahoma",
                                                                  "Missouri", "Arkansas", "Ohio", "Kentucky",
                                                                  "North Carolina", "West Virgina", "Delaware","Mississippi",
                                                                  "California","Utah","Hawaii", "Total"])]

# Count of Registered Motorcycles calculated for all law groups:

legal_reg_2019 = legal_reg.iloc[0, 1]
legal_reg_2020 = legal_reg.iloc[0,2]
legal_reg_2021 = legal_reg.iloc[0,3]
legal_reg_total = legal_reg_2019 + legal_reg_2020 + legal_reg_2021

somewhat_legal_reg_2019 = somewhat_legal_reg.iloc[:, 1].sum()
somewhat_legal_reg_2020 = somewhat_legal_reg.iloc[:,2].sum()
somewhat_legal_reg_2021 = somewhat_legal_reg.iloc[:,3].sum()
somewhat_legal_reg_total = somewhat_legal_reg_2019 + somewhat_legal_reg_2020 + somewhat_legal_reg_2021

not_mentioned_reg_2019 = legal_reg.iloc[:, 1].sum()
not_mentioned_reg_2020 = legal_reg.iloc[:,2].sum()
not_mentioned_reg_2021 = legal_reg.iloc[:,3].sum()
not_mentioned_reg_total = not_mentioned_reg_2019 + not_mentioned_reg_2020 + not_mentioned_reg_2021

illegal_reg_2019 = illegal_reg.iloc[:, 1].sum()
illegal_reg_2020 = illegal_reg.iloc[:,2].sum()
illegal_reg_2021 = illegal_reg.iloc[:,3].sum()
illegal_reg_total = illegal_reg_2019 + illegal_reg_2020 + illegal_reg_2021





In [ ]:
# Motorcycle type affects safety of lane splitting (Calvin)

In [ ]:
# Motorcycle laws affects fewer crashes (Rail)

In [ ]:
# Motorcycle laws affects deadly crashes (Jack)



In [ ]:
# Motorcycle  laws affects injuries (Rachael)

In [ ]:
#compare UT years accidents with chart 

In [ ]:
#compare ID years accidents with chart 

In [ ]:
#compare UT & ID to CA with chart

In [ ]:
#compare COLL types 